__Reconstructing the path__

In this assignment you will use Spark to compute the shortest path between two vertices. In the video, you have learned how to compute the distances between a source vertex and all other vertices in a graph. Now, your task is to reconstruct the shortest path, that is a sequence of vertices connected by the edges.

Dataset location: /data/twitter/twitter_sample_small.txt

Format: user_id \t follower_id

You can start with the code described in "Starter..." (see the next self-reading).

Your task is to find the shortest path between vertices 12 and 34. In case of multiple shortest paths (that is, disjoint paths with the same length), any will suffice. Output format is the sequence of vertices, delimited by a comma, without spaces. For example, the path “12 -> 42 -> 34” should be printed as:

Hint: before submitting, check your stopping criteria. In BFS, the search was exhaustive, and in this task your program may terminate earlier, thus saving some precious time.

The result on the sample dataset:

12,422,53,52,107,20,23,274,34

In [2]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("shortest path assignment").setMaster("local"))

In [3]:
def parse_edge(s):
  user, follower = s.split("\t")
  return (int(user), int(follower))

def step(item):
  prev_v, prev_d, next_v = item[0], item[1][0], item[1][1]
  return (next_v, prev_d + 1)

def complete(item):
  v, old_d, new_d = item[0], item[1][0], item[1][1]
  return (v, old_d if old_d is not None else new_d)

n = 400  # number of partitions
edges = sc.textFile("/data/twitter/twitter_sample_small.txt").map(parse_edge).cache()
forward_edges = edges.map(lambda e: (e[1], e[0])).partitionBy(n).persist()

x = 12
d = 0
distances = sc.parallelize([(x, d)]).partitionBy(n)
while True:
  candidates = distances.join(forward_edges, n).map(step)
  new_distances = distances.fullOuterJoin(candidates, n).map(complete, True).persist()
  count = new_distances.filter(lambda i: i[1] == d + 1).count()
  if count > 0:
    d += 1
    distances = new_distances
  else:
    break

In [6]:
# 15min
distances.take(8)

[(12, 0), (13, 4), (15, 5), (15, 5), (15, 5), (15, 5), (15, 5), (15, 5)]

simple topology
```
  6
 /\
1--2
|  |
3--4
 \/
 5
```

In [7]:
edg = sc.parallelize([(1,2),(1,3),(2,1),(2,4),(3,1),(3,4),(4,3),(4,2),(3,5),(5,3),(4,5),(5,4),(6,1),(1,6),(6,2),(2,6)])

In [10]:
fwd_edg = edg.map(lambda e: (e[1], e[0])) # swap

In [11]:
edg.collect()

[(1, 2),
 (1, 3),
 (2, 1),
 (2, 4),
 (3, 1),
 (3, 4),
 (4, 3),
 (4, 2),
 (3, 5),
 (5, 3),
 (4, 5),
 (5, 4),
 (6, 1),
 (1, 6),
 (6, 2),
 (2, 6)]

In [12]:
fwd_edg.collect()

[(2, 1),
 (3, 1),
 (1, 2),
 (4, 2),
 (1, 3),
 (4, 3),
 (3, 4),
 (2, 4),
 (5, 3),
 (3, 5),
 (5, 4),
 (4, 5),
 (1, 6),
 (6, 1),
 (2, 6),
 (6, 2)]

In [148]:
src = (5, 0)
dist = sc.parallelize([src])
dist.collect()

[(5, 0)]

In [149]:
dist_fwd_join = dist.join(fwd_edg)
dist_fwd_join.collect()

[(5, (0, 3)), (5, (0, 4))]

In [150]:
def step2(item): v, d, nv = item[0], item[1][0], item[1][1]; return (nv, (v, d))

In [152]:
dist_fwd_join.map(step2).collect()

[(3, (5, 0)), (4, (5, 0))]

In [153]:
cand = dist_fwd_join.map(step2)
cand.collect()

[(3, (5, 0)), (4, (5, 0))]

In [154]:
new_cand = cand.join(fwd_edg)
new_cand.collect()

[(3, ((5, 0), 1)),
 (3, ((5, 0), 4)),
 (3, ((5, 0), 5)),
 (4, ((5, 0), 2)),
 (4, ((5, 0), 3)),
 (4, ((5, 0), 5))]

In [155]:
new_cand.collect()[0][1][0][0]

5

In [163]:
def complete2(item): 
    v, d, nv = item[0], item[1][0], item[1][1]; 
    path = d + (v,)
    return (nv, (path), v)

In [164]:
new_dist = new_cand.map(complete2)
new_dist.collect()

[(1, (5, 0, 3), 3),
 (4, (5, 0, 3), 3),
 (5, (5, 0, 3), 3),
 (2, (5, 0, 4), 4),
 (3, (5, 0, 4), 4),
 (5, (5, 0, 4), 4)]

In [166]:
cand2 = new_dist.join(fwd_edg)
cand2.collect()

[(4, ((5, 0, 3), 2)),
 (4, ((5, 0, 3), 3)),
 (4, ((5, 0, 3), 5)),
 (1, ((5, 0, 3), 2)),
 (1, ((5, 0, 3), 3)),
 (1, ((5, 0, 3), 6)),
 (5, ((5, 0, 3), 3)),
 (5, ((5, 0, 3), 4)),
 (5, ((5, 0, 4), 3)),
 (5, ((5, 0, 4), 4)),
 (2, ((5, 0, 4), 1)),
 (2, ((5, 0, 4), 4)),
 (2, ((5, 0, 4), 6)),
 (3, ((5, 0, 4), 1)),
 (3, ((5, 0, 4), 4)),
 (3, ((5, 0, 4), 5))]

In [169]:
dist2 = cand2.map(complete2)
dist2.collect()

[(2, (5, 0, 3, 4), 4),
 (3, (5, 0, 3, 4), 4),
 (5, (5, 0, 3, 4), 4),
 (2, (5, 0, 3, 1), 1),
 (3, (5, 0, 3, 1), 1),
 (6, (5, 0, 3, 1), 1),
 (3, (5, 0, 3, 5), 5),
 (4, (5, 0, 3, 5), 5),
 (3, (5, 0, 4, 5), 5),
 (4, (5, 0, 4, 5), 5),
 (1, (5, 0, 4, 2), 2),
 (4, (5, 0, 4, 2), 2),
 (6, (5, 0, 4, 2), 2),
 (1, (5, 0, 4, 3), 3),
 (4, (5, 0, 4, 3), 3),
 (5, (5, 0, 4, 3), 3)]

In [170]:
cand3 = dist2.join(fwd_edg)
cand3.collect()

[(5, ((5, 0, 3, 4), 3)),
 (5, ((5, 0, 3, 4), 4)),
 (5, ((5, 0, 4, 3), 3)),
 (5, ((5, 0, 4, 3), 4)),
 (1, ((5, 0, 4, 2), 2)),
 (1, ((5, 0, 4, 2), 3)),
 (1, ((5, 0, 4, 2), 6)),
 (1, ((5, 0, 4, 3), 2)),
 (1, ((5, 0, 4, 3), 3)),
 (1, ((5, 0, 4, 3), 6)),
 (6, ((5, 0, 3, 1), 1)),
 (6, ((5, 0, 3, 1), 2)),
 (6, ((5, 0, 4, 2), 1)),
 (6, ((5, 0, 4, 2), 2)),
 (2, ((5, 0, 3, 4), 1)),
 (2, ((5, 0, 3, 4), 4)),
 (2, ((5, 0, 3, 4), 6)),
 (2, ((5, 0, 3, 1), 1)),
 (2, ((5, 0, 3, 1), 4)),
 (2, ((5, 0, 3, 1), 6)),
 (3, ((5, 0, 3, 4), 1)),
 (3, ((5, 0, 3, 4), 4)),
 (3, ((5, 0, 3, 4), 5)),
 (3, ((5, 0, 3, 1), 1)),
 (3, ((5, 0, 3, 1), 4)),
 (3, ((5, 0, 3, 1), 5)),
 (3, ((5, 0, 3, 5), 1)),
 (3, ((5, 0, 3, 5), 4)),
 (3, ((5, 0, 3, 5), 5)),
 (3, ((5, 0, 4, 5), 1)),
 (3, ((5, 0, 4, 5), 4)),
 (3, ((5, 0, 4, 5), 5)),
 (4, ((5, 0, 3, 5), 2)),
 (4, ((5, 0, 3, 5), 3)),
 (4, ((5, 0, 3, 5), 5)),
 (4, ((5, 0, 4, 5), 2)),
 (4, ((5, 0, 4, 5), 3)),
 (4, ((5, 0, 4, 5), 5)),
 (4, ((5, 0, 4, 2), 2)),
 (4, ((5, 0, 4, 2), 3)),


In [171]:
dist3 = cand3.map(complete2)
dist3.collect()

[(3, (5, 0, 3, 4, 5), 5),
 (4, (5, 0, 3, 4, 5), 5),
 (3, (5, 0, 4, 3, 5), 5),
 (4, (5, 0, 4, 3, 5), 5),
 (2, (5, 0, 4, 2, 1), 1),
 (3, (5, 0, 4, 2, 1), 1),
 (6, (5, 0, 4, 2, 1), 1),
 (2, (5, 0, 4, 3, 1), 1),
 (3, (5, 0, 4, 3, 1), 1),
 (6, (5, 0, 4, 3, 1), 1),
 (1, (5, 0, 3, 1, 6), 6),
 (2, (5, 0, 3, 1, 6), 6),
 (1, (5, 0, 4, 2, 6), 6),
 (2, (5, 0, 4, 2, 6), 6),
 (1, (5, 0, 3, 4, 2), 2),
 (4, (5, 0, 3, 4, 2), 2),
 (6, (5, 0, 3, 4, 2), 2),
 (1, (5, 0, 3, 1, 2), 2),
 (4, (5, 0, 3, 1, 2), 2),
 (6, (5, 0, 3, 1, 2), 2),
 (1, (5, 0, 3, 4, 3), 3),
 (4, (5, 0, 3, 4, 3), 3),
 (5, (5, 0, 3, 4, 3), 3),
 (1, (5, 0, 3, 1, 3), 3),
 (4, (5, 0, 3, 1, 3), 3),
 (5, (5, 0, 3, 1, 3), 3),
 (1, (5, 0, 3, 5, 3), 3),
 (4, (5, 0, 3, 5, 3), 3),
 (5, (5, 0, 3, 5, 3), 3),
 (1, (5, 0, 4, 5, 3), 3),
 (4, (5, 0, 4, 5, 3), 3),
 (5, (5, 0, 4, 5, 3), 3),
 (2, (5, 0, 3, 5, 4), 4),
 (3, (5, 0, 3, 5, 4), 4),
 (5, (5, 0, 3, 5, 4), 4),
 (2, (5, 0, 4, 5, 4), 4),
 (3, (5, 0, 4, 5, 4), 4),
 (5, (5, 0, 4, 5, 4), 4),
 (2, (5, 0, 

In [172]:
dist3.lookup(6)

[(5, 0, 4, 2, 1), (5, 0, 4, 3, 1), (5, 0, 3, 4, 2), (5, 0, 3, 1, 2)]

several searching throw neigbours table with different key.

In [173]:
dist2.lookup(6)

[(5, 0, 3, 1), (5, 0, 4, 2)]